In [41]:
import os
import midas_touch2 as mt2
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
import matplotlib.pyplot as plt
import stock_mapping as sm
import warnings
import asyncio
import twap
import time
from datetime import datetime, timedelta
from class_mapping import StockSymbolMapper
pd.options.display.max_columns = 100000
warnings.filterwarnings('ignore')

if not mt5.initialize():
    print("initialize() failed")
else:
    print("MT5 successfully initialised.\n")

MT5 successfully initialised.



In [3]:
def monitor_portfolio(stock_symbols, weights):
    if not mt5.initialize():
        print("initialize() failed, error code =", mt5.last_error())
        return
    all_positions = []
    
    for symbol in stock_symbols:
        positions = mt5.positions_get(symbol=symbol)
        if positions is None:
            print(f"No positions for {symbol}, error code={mt5.last_error()}")
        elif len(positions) > 0:
            all_positions.extend(list(positions))
    if all_positions:
        df = pd.DataFrame(all_positions, columns=all_positions[0]._asdict().keys())
        df['time'] = pd.to_datetime(df['time'], unit='s')
        df = df.drop(['time_msc', 'time_update', 'time_update_msc', 'identifier', 'reason', 
                      'sl', 'tp', 'swap', 'comment', 'external_id'], axis=1)
        df['position_change'] = (df['price_current'] / df['price_open']) - 1
        df['value'] = df['volume'] * df['price_open']
        df['weights'] = df['symbol'].map(weights)
        df['portfolio_change_position'] = df['weights'] * df['position_change']
        #print(df)
    else:
        print("No open positions for any of the specified symbols.")
    mt5.shutdown()
    total_portfolio_change_position = df['portfolio_change_position'].sum()
    print(f"Total portfolio change position: {total_portfolio_change_position:.4%}")
    return total_portfolio_change_position

In [14]:
latest_weights

HLFG       0.238689
PBBANK    -0.000000
FFB        0.000000
MAYBANK    0.095896
MSM       -0.000000
HARTA      0.665415
Name: 2023-11-30, dtype: float64

In [47]:
twap.TWAPStrategy2(symbol = "PERTAMA", 
                   action = "Buy", 
                   execPeriod = 60*60*10, 
                   execMode = 'Qty', 
                   execUnits = 21600, 
                   targetTWAP= 2.89, 
                   slippage = 0.02, 
                   qtyRatio = 0.2, 
                   orderInterval = 5, 
                   magic = 6666, 
                   comment = "")


Timestamp: 1701417581.7895715, Matched Qty: 17200.0, Matched Price: 2.9, Transacted Qty: 17200.0, Transacted Value: 49880.0, Balance: 4400.0
Timestamp: 1701417584.847098, Matched Qty: 4400.0, Matched Price: 2.9, Transacted Qty: 21600.0, Transacted Value: 62640.0, Balance: 0.0


In [ ]:
TWAPStrategy2("PERTAMA", "Sell", 3600, "Qty", 1600.0, 2.85, 0.02, 0.02, 10, 0.5, 1.5, 20231129, "TWAP")

In [40]:
stock_symbols = ['HLFG','PBBANK','FFB','MAYBANK','MSM','HARTA']
weights = latest_weights
if not mt5.initialize():
    print("initialize() failed, error code =", mt5.last_error())
from_date=datetime.now()
to_date=datetime.now()+timedelta(days=1)
all_deals = []
all_deals = []
for symbol in stock_symbols:
    deals= mt5.history_orders_get(from_date, to_date)
    if deals is None:
        print(f"No deals for {symbol}, error code={mt5.last_error()}")
    elif len(deals) > 0:
        all_deals.extend(list(deals))

if all_deals:
    df = pd.DataFrame(all_deals, columns=all_deals[0]._asdict().keys())
    df['time_done'] = pd.to_datetime(df['time_done'], unit='s')
    # df = df.drop(['time_msc', 'time_update', 'time_update_msc', 'identifier', 'reason', 
    #                 'sl', 'tp', 'swap', 'comment', 'external_id'], axis=1)
# ['ticket','order','time_msc','type','entry',
#        'reason', 'volume', 'price', 'commission', 'swap',
#        'profit', 'fee', 'symbol', 'comment', 'external_id'],

df

,ticket,time_setup,time_setup_msc,time_done,time_done_msc,time_expiration,type,type_time,type_filling,state,magic,position_id,position_by_id,reason,volume_initial,volume_current,price_open,sl,tp,price_current,price_stoplimit,symbol,comment,external_id
0,582139,1701430155,1701430155645,2023-12-01 11:29:39,1701430179786,0,2,0,2,4,8888,582139,0,3,200.0,0.0,16.50,0.0,0.0,16.500,0.0,HLFG,Entry,
1,582140,1701430155,1701430155728,2023-12-01 11:29:44,1701430184153,0,2,0,2,4,8888,582140,0,3,200.0,0.0,9.03,0.0,0.0,9.030,0.0,MAYBANK,Entry,
2,582141,1701430155,1701430155773,2023-12-01 11:29:23,1701430163182,0,2,0,2,4,8888,582141,0,3,5500.0,0.0,2.46,0.0,0.0,2.450,0.0,HARTA,Entry,
3,582146,1701430276,1701430276931,2023-12-01 11:31:16,1701430276931,0,1,0,2,4,0,580643,0,0,10000.0,0.0,0.00,0.0,0.0,4.880,0.0,YNHPROP,,
4,582147,1701430277,1701430277458,2023-12-01 11:31:17,1701430277458,0,1,0,2,4,0,582141,0,0,5500.0,0.0,0.00,0.0,0.0,2.440,0.0,HARTA,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,582757,1701444365,1701444365831,2023-12-01 15:26:08,1701444368896,0,2,0,2,4,8888,582757,0,3,200.0,0.0,9.00,0.0,0.0,9.000,0.0,MAYBANK,Entry,
104,582758,1701444365,1701444365915,2023-12-01 15:26:46,1701444406877,0,2,0,2,4,8888,582758,0,3,5500.0,0.0,2.46,0.0,0.0,2.450,0.0,HARTA,Entry,
105,582787,1701444790,1701444790223,2023-12-01 15:33:10,1701444790223,0,0,0,2,4,0,582787,0,0,200000.0,0.0,0.00,0.0,0.0,0.035,0.0,XOXNET,,
106,582789,1701444819,1701444819964,2023-12-01 15:33:39,1701444819964,0,0,0,2,4,0,582789,0,0,200000.0,0.0,0.00,0.0,0.0,2.850,0.0,PERTAMA,,


In [ ]:
['ticket', 'order', 'time', 'time_msc', 'type', 'entry', 'magic',
'position_id', 'reason', 'volume', 'price', 'commission', 'swap',
'profit', 'fee', 'symbol', 'comment', 'external_id'],

In [ ]:
def compile_portfolio_data(allocation_df, starting_cash, current_share_value):
    purchase_cost = (allocation_df['Share Price'] * allocation_df['Entry Units']).sum()
    sale_proceed = (allocation_df['Share Price'] * allocation_df['Exit Units']).sum()
    cash_value = starting_cash - purchase_cost + sale_proceed
    portfolio_value = current_share_value
    total_value = portfolio_value + cash_value  # adjusted to account for the initial capital
    portfolio_data = {
        'Starting Capital': [starting_cash],
        'Purchase Cost': [purchase_cost],
        'Sale Proceed': [sale_proceed],
        'Cash Value': [cash_value],
        'Portfolio Value': [portfolio_value],
        'Total Value': [total_value]
    }
    portfolio_df = pd.DataFrame(portfolio_data)
    return portfolio_df

In [7]:
yf_symbols =['5227.KL','5176.KL','5212.KL','5106.KL','5110.KL','5307.KL',
            '6947.KL','6012.KL','1155.KL','1295.KL','1066.KL','1015.KL', #'6888.KL',
            '1818.KL','2488.KL','5185.KL','5258.KL','6139.KL','9296.KL','1082.KL',
            '6459.KL','3034.KL','5273.KL','4731.KL','7172.KL','5284.KL','9822.KL',
            '3476.KL','7100.KL','2852.KL','7095.KL','4758.KL','5302.KL','0151.KL',
            '7233.KL','5271.KL','5916.KL','0215.KL','1368.KL','5000.KL','7231.KL',
            '7034.KL','8907.KL','6556.KL','7245.KL','6971.KL','0223.KL', #'7195.KL',
            '8052.KL','7609.KL','7222.KL','5673.KL','7229.KL','0099.KL','7133.KL',
            '5568.KL','5192.KL','7029.KL','7197.KL','7004.KL','3816.KL','5014.KL',
            '5246.KL','6521.KL','2062.KL','7293.KL','5199.KL','5141.KL','3042.KL',
            '5071.KL','4324.KL','5132.KL','5249.KL','5200.KL','8206.KL','5401.KL',
            '3158.KL','5053.KL','5236.KL','5038.KL','6114.KL','7249.KL','3182.KL',
            '4715.KL','5296.KL','7084.KL','4588.KL','4006.KL','5248.KL','3301.KL',
            '4162.KL','1619.KL','5102.KL','7052.KL','5306.KL','8532.KL','3859.KL',
            '6599.KL','5983.KL','5517.KL','7237.KL','0186.KL','5202.KL','7087.KL',
            '5908.KL','9172.KL','7216.KL','7103.KL','6432.KL','7252.KL','9288.KL',
            '7107.KL','7085.KL','7230.KL','0097.KL','0208.KL','5005.KL','0128.KL',
            '7204.KL','5292.KL','7160.KL','5301.KL','5309.KL','5286.KL','0146.KL',
            '7022.KL','5347.KL','6742.KL','4677.KL','5209.KL','3069.KL','5168.KL',
            '5878.KL','7153.KL','7090.KL','7148.KL','0001.KL','1961.KL','5222.KL',
            '2291.KL','5254.KL','5126.KL','5027.KL','5138.KL','5012.KL','9059.KL',
            '6262.KL','3336.KL','5211.KL','7161.KL','5293.KL']
file_path = r'C:\Users\jeffnsy\Desktop\Python codes\midas_touch2\stock_mapping.xlsx'
mt5_symbol = sm.get_ticker_list(yf_symbols, file_path=file_path)
print(mt5_symbol)

['IGBREIT', 'SUNREIT', 'PAVREIT', 'AXREIT', 'UOAREIT', 'AMEREIT', 'CDB', 'MAXIS', 'MAYBANK', 'PBBANK', 'RHBBANK', 'AMBANK', 'BURSA', 'ABMB', 'AFFIN', 'BIMB', 'TAKAFUL', 'RCECAP', 'HLFG', 'MNRB', 'HAPSENG', 'CHINHIN', 'SCIENTX', 'PMBTECH', 'LCTITAN', 'SAM', 'KSENG', 'UCHITEC', 'CMSB', 'PIE', 'ANCOMNY', 'ATECH', 'KGB', 'DUFU', 'PECCA', 'MSC', 'SLVEST', 'EDGENTA', 'HUMEIND', 'WELLCAL', 'TGUAN', 'EG', 'ANNJOO', 'CITAGLB', 'KOBAY', 'SAMAIDEN', 'CGB', 'AJIYA', 'IMASPRO', 'JSB', 'FAVCO', 'SCICOM', 'ULICORP', 'APB', 'KSSC', 'MASTER', 'GESHEN', 'MCEHLDG', 'MISC', 'AIRPORT', 'WPRTS', 'SURIA', 'HARBOUR', 'YINSON', 'HIBISCS', 'DAYANG', 'PETRONM', 'COASTAL', 'HENGYUAN', 'DELEUM', 'IOIPG', 'UOADEV', 'ECOWLD', 'TROP', 'YNHPROP', 'OSK', 'MATRIX', 'KSL', 'MKH', 'EWEIN', 'GENTING', 'GENM', 'MRDIY', 'QL', 'UMW', 'ORIENT', 'BAUTO', 'HLIND', 'BAT', 'DRBHCOM', 'GCB', 'PADINI', 'FFB', 'SINOTOP', 'MAGNUM', 'AEON', 'MBMR', 'SHANG', 'PWROOT', 'PTRANS', 'MSM', 'MAGNI', 'DKSH', 'FPI', 'KAWAN', 'SPRITZER', 'APOLLO

In [6]:
yf_symbols = ["1082.KL","1295.KL","5306.KL","1155.KL","5202.KL","5168.KL"]#'7195.KL',
symbols = [symbol + "SE" for symbol in yf_symbols]
api_token = "65090d11a67b45.83965572"
data = mt2.download_stocks(symbols, "d", api_token, num_days=20)
price_df = pd.DataFrame(data)
#rename columns
mapping_file_path = "C:/Users/jeffnsy/Desktop/Python codes/midas_touch2/stock_mapping.xlsx"
mt5_symbol = sm.get_ticker_list(yf_symbols, file_path=mapping_file_path)
price_df.columns = mt5_symbol
#price_df.columns = yf_symbols
weights_df, latest_weights = mt2.run_portfolio(price_df, alpha_n=5)
price_df.tail()

,HLFG,PBBANK,FFB,MAYBANK,MSM,HARTA
time,,,,,,
2023-11-24,16.48,4.25,1.29,9.09,1.38,2.52
2023-11-27,16.54,4.24,1.29,9.06,1.46,2.50
2023-11-28,16.54,4.22,1.31,9.05,1.39,2.53
2023-11-29,16.40,4.24,1.36,9.04,1.40,2.45
2023-11-30,16.10,4.27,1.41,8.98,1.42,2.40


In [5]:
latest_weights

NameError: name 'latest_weights' is not defined

In [5]:
# file_path = r'C:\Users\kkyao\midas_touch2\stock_mapping.xlsx'
# mapper = StockSymbolMapper(file_path)
# output = mapper.find_opposite(yf_symbols)
# print(output)

In [6]:
###dummy weights to test
import random
def generate_random_series(tickers, seed=None):
    if seed is not None:
        random.seed(seed)
    random_values = [random.random() for _ in tickers]
    total = sum(random_values)
    normalized_values = [value / total for value in random_values]
    return pd.Series(normalized_values, index=tickers)
latest_weights = generate_random_series(mt5_symbol)
latest_weights

MAYBANK    0.067939
YNHPROP    0.062925
IJM        0.122519
KMLOONG    0.138098
CTOS       0.049629
AMEREIT    0.030785
KPJ        0.082356
SURIA      0.097402
YTLPOWR    0.109022
AXIATA     0.061265
PADINI     0.080730
YINSON     0.097331
dtype: float64

In [7]:
total_investment = 100000 #TRY 15K 10K 5K
previous_allocation_df = mt2.get_folder(folder_name='daily_allocation')
previous_portfolio_df = mt2.get_folder(folder_name='daily_portfolio')
if previous_allocation_df is not None and previous_portfolio_df is not None:
    # Process the existing CSV data
    prev_holding = previous_allocation_df['Holding Units']
    prev_entry = previous_allocation_df['Entry Units']
    prev_exit = previous_allocation_df['Exit Units']
    total_investment = abs(previous_portfolio_df['Total Value'].iloc[-1])
    allocation_df = mt2.calculate_stock_allocation(total_investment, latest_weights, price_df)
    allocation_df['Holding Units'] = prev_holding + prev_entry - prev_exit
else: #first run
    allocation_df = mt2.calculate_stock_allocation(total_investment, latest_weights, price_df)
    # Handle the initial run scenario
    print("This is the initial run. No previous allocation data available.")
    allocation_df['Holding Units'] = 0

# Calculate Entry and Exit Units
allocation_df['Entry Units'] = allocation_df.apply(
    lambda row: max(row['Allocated Units'] - row['Holding Units'], 0), axis=1
)
allocation_df['Exit Units'] = allocation_df.apply(
    lambda row: max(row['Holding Units'] - row['Allocated Units'], 0), axis=1
)
numeric_cols = allocation_df.select_dtypes(include=['number']).astype(float)

for col in numeric_cols.columns:
    allocation_df[col] = numeric_cols[col]

########################---async start here---#########################
print(allocation_df)
order_ids = mt2.execute_trades_from_data(allocation_df)
time.sleep(5)
orders_list = [mt2.check_order_status(order_id) for order_id in order_ids]
print(f'orders_list: {orders_list}')
deleted_volumes = mt2.delete_orders(orders_list)
print(f'deleted_volumes: {deleted_volumes}')
########################---end here----#################################
for order in deleted_volumes:
    symbol = order['symbol']
    volume_filled = order['volume_filled']
    if order['direction'] == mt5.ORDER_TYPE_BUY_LIMIT:
        allocation_df.loc[allocation_df['Share Symbol'] == symbol, 'Entry Units'] = volume_filled
    else: 
        allocation_df.loc[allocation_df['Share Symbol'] == symbol, 'Exit Units'] = volume_filled

    #allocation_df.loc[allocation_df['Share Symbol'] == symbol, 'Allocated Units'] = volume_filled
# async def execute_and_monitor(allocation_df, timeout=60):
#     print(allocation_df)
#     order_ids = mt2.execute_trades_from_data(allocation_df)
#     print(f'Waiting for {timeout/60} minutes..')
#     await asyncio.sleep(timeout)
#     orders_list = [mt2.check_order_status(order_id) for order_id in order_ids]  # Renamed from orders_list
#     print(f'orders_list: {orders_list}')
#     resubmitted_orders = mt2.delete_and_resubmit_orders(orders_list)  # Renamed from resubmitted_orders
#     print(f'resubmitted_orders: {resubmitted_orders}')
#     return orders_list, resubmitted_orders  # Return the renamed variables

# # Execute the async function and capture the returned values with the new variable names
# orders_list, resubmitted_orders = asyncio.run(execute_and_monitor(allocation_df, timeout=90))

# Calculate the portfolio value
entry_unit = allocation_df['Entry Units']
exit_unit = allocation_df['Exit Units']
share_price = allocation_df['Share Price']
allocated_unit = allocation_df['Allocated Units']
if previous_allocation_df is not None and previous_portfolio_df is not None: #2nd run onwards
    starting_cash = previous_portfolio_df['Cash Value'].iloc[-1]
    current_share_value = ((entry_unit*share_price)-(exit_unit*share_price)).sum()
    current_share_value += previous_portfolio_df['Portfolio Value'].iloc[-1]

else: #first run
    starting_cash = total_investment
    current_share_value = (entry_unit*share_price).sum()

portfolio_df = mt2.compile_portfolio_data(allocation_df, starting_cash, current_share_value)

mt2.save_df_to_csv(portfolio_df, 
                   folder_name='daily_portfolio', 
                   file_name='balance',
                   append=True)
mt2.save_df_to_csv(allocation_df, 
                   folder_name='daily_allocation', 
                   file_name='allocation')


Folder not found: daily_allocation
Folder not found: daily_portfolio
This is the initial run. No previous allocation data available.
   Share Symbol  Share Price  Allocated Units   Weights  Holding Units  \
0       MAYBANK         9.08            700.0  0.067939            0.0   
1       YNHPROP         5.07           1200.0  0.062925            0.0   
2           IJM         1.89           6400.0  0.122519            0.0   
3       KMLOONG         1.89           7300.0  0.138098            0.0   
4          CTOS         1.43           3400.0  0.049629            0.0   
5       AMEREIT         1.25           2400.0  0.030785            0.0   
6           KPJ         1.25           6500.0  0.082356            0.0   
7         SURIA         1.70           5700.0  0.097402            0.0   
8       YTLPOWR         2.20           4900.0  0.109022            0.0   
9        AXIATA         2.40           2500.0  0.061265            0.0   
10       PADINI         3.93           2000.0  0.0807

In [8]:
deleted_volumes

[{'symbol': 'MAYBANK', 'volume_filled': 0.0, 'direction': 2},
 {'symbol': 'YNHPROP', 'volume_filled': 0.0, 'direction': 2},
 {'symbol': 'IJM', 'volume_filled': 0.0, 'direction': 2},
 {'symbol': 'KMLOONG', 'volume_filled': 0.0, 'direction': 2},
 {'symbol': 'CTOS', 'volume_filled': 0.0, 'direction': 2},
 {'symbol': 'AMEREIT', 'volume_filled': 0.0, 'direction': 2},
 {'symbol': 'KPJ', 'volume_filled': 0.0, 'direction': 2},
 {'symbol': 'SURIA', 'volume_filled': 0.0, 'direction': 2},
 {'symbol': 'YTLPOWR', 'volume_filled': 0.0, 'direction': 2},
 {'symbol': 'AXIATA', 'volume_filled': 0.0, 'direction': 2},
 {'symbol': 'PADINI', 'volume_filled': 0.0, 'direction': 2},
 {'symbol': 'YINSON', 'volume_filled': 0.0, 'direction': 2}]

In [9]:
print(f'starting_cash: {starting_cash}')
print(f'current_share_value: {current_share_value}')

starting_cash: 100000
current_share_value: 0.0


In [10]:
portfolio_df

,Starting Capital,Purchase Cost,Sale Proceed,Cash Value,Portfolio Value,Total Value
0,100000,0.0,0.0,100000.0,0.0,100000.0


In [11]:
allocation_df

,Share Symbol,Share Price,Allocated Units,Weights,Holding Units,Entry Units,Exit Units
0,MAYBANK,9.08,700.0,0.067939,0.0,0.0,0.0
1,YNHPROP,5.07,1200.0,0.062925,0.0,0.0,0.0
2,IJM,1.89,6400.0,0.122519,0.0,0.0,0.0
3,KMLOONG,1.89,7300.0,0.138098,0.0,0.0,0.0
4,CTOS,1.43,3400.0,0.049629,0.0,0.0,0.0
5,AMEREIT,1.25,2400.0,0.030785,0.0,0.0,0.0
6,KPJ,1.25,6500.0,0.082356,0.0,0.0,0.0
7,SURIA,1.70,5700.0,0.097402,0.0,0.0,0.0
8,YTLPOWR,2.20,4900.0,0.109022,0.0,0.0,0.0
9,AXIATA,2.40,2500.0,0.061265,0.0,0.0,0.0


In [12]:
price_df

,MAYBANK,YNHPROP,IJM,KMLOONG,CTOS,AMEREIT,KPJ,SURIA,YTLPOWR,AXIATA,PADINI,YINSON
time,,,,,,,,,,,,
2023-10-16,8.98,5.10,1.90,1.8800,1.43,1.26,1.20,1.59,1.9689,2.38,3.93,2.47
2023-10-17,9.00,5.10,1.89,1.8508,1.42,1.26,1.20,1.60,1.9689,2.43,3.94,2.49
2023-10-18,8.95,5.10,1.89,1.8605,1.45,1.26,1.19,1.60,1.9590,2.43,3.94,2.39
2023-10-19,8.96,5.10,1.83,1.8605,1.43,1.25,1.20,1.59,1.9295,2.42,3.93,2.42
2023-10-20,8.99,5.10,1.81,1.8995,1.43,1.25,1.20,1.60,1.9295,2.43,3.96,2.43
2023-10-23,8.99,5.10,1.82,1.8605,1.41,1.26,1.20,1.59,1.9197,2.41,3.94,2.39
2023-10-24,8.96,5.10,1.86,1.8800,1.42,1.27,1.24,1.58,1.9689,2.41,4.00,2.40
2023-10-25,9.00,5.10,1.86,1.8900,1.43,1.25,1.24,1.58,1.9689,2.41,3.98,2.46
2023-10-26,9.00,5.10,1.90,1.8900,1.45,1.27,1.25,1.58,2.0083,2.35,3.98,2.40
